In [1]:
repo_dir = "Repos"  
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-vgg19" / "src" / "vgg19"))
sys.path.insert(0, str(Path.home() / repo_dir / "RETFound_MAE"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-exec" / "models" / "vgg19")) 

In [2]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2025-03-31 15:52:35.890834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-31 15:52:35.890879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-31 15:52:35.895324: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-31 15:52:35.908772: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-31 15:52:37.394817: W tensorflow/compiler/tf2

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

You are already logged in.


In [4]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-03-31 15:52:39,506 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install


In [5]:
datasets = [
    '4-4116', # Selected images for training
    '4-411G', # Selected images for testing
    '2-7P5P', # Full multimodal dataset
    ]

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)

workflow_instance = EA.create_workflow(
    name="Multimodal workflow",
    workflow_type="Multimodal workflow"
)

config = ExecutionConfiguration(
    datasets=to_be_download,
    assets = ['2-4JR6',],
    workflow=workflow_instance,
    description="Instance of cropping multimodal images.")

execution = EA.create_execution(config)

2025-03-31 15:52:40,514 - INFO - Materialize bag 4-4116... 
2025-03-31 15:52:41,450 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-03-31 15:52:42,218 - INFO - Loading /data/4-4116_5a611f3561ac538bae3767ae4c1bafa1533b9722fca8a1fc78085f422c5290f6/Dataset_4-4116
2025-03-31 15:52:42,830 - INFO - Creating new database for dataset: 4-4116 in /data/nguyent8/EyeAI_working/4-4116@330-B1Z0-2NHT.db
2025-03-31 15:52:42,831 - INFO - Materialize bag 4-411G... 
2025-03-31 15:52:43,970 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-03-31 15:52:44,832 - INFO - Loading /data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d70f7b3bfd0a0cd3931d4e2edbade797d/Dataset_4-411G
2025-03-31 15:52:45,231 - INFO - Creating new database for dataset: 4-411G in /data/nguyent8/EyeAI_working/4-411G@330-B1Z0-GC2Y.db
2025-03-31 15:52:45,232 - INFO - Materialize bag 2-7P5P... 
2025-03-31 15:52:46,348 - WARNING - nbstripout is not installed in r

In [6]:
print(execution)

caching_dir: /data
_working_dir: /data/nguyent8/EyeAI_working
execution_rid: 4-S524
workflow_rid: 4-S522
asset_paths: [PosixPath('/data/nguyent8/EyeAI_working/4-S524/asset/optic_disk_crop_model.hdf5')]
configuration: datasets=[DatasetSpec(rid='4-4116', materialize=True, version=DatasetVersion(major=2, minor=1, patch=0)), DatasetSpec(rid='4-411G', materialize=True, version=DatasetVersion(major=2, minor=1, patch=0)), DatasetSpec(rid='2-7P5P', materialize=True, version=DatasetVersion(major=2, minor=1, patch=0))] assets=['2-4JR6'] workflow=Workflow(name='Multimodal workflow', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/bb26e92ce4746a3637372d73a887a36bbdde2aa9--/notebooks/crop_multimodal_images.ipynb', workflow_type='Multimodal workflow', version=None, description='', rid=None, checksum='e69de29bb2d1d6434b8b29ae775ad8c2e48c5391') parameters={} description='Instance of cropping multimodal images.' argv=['/home/nguyent8/.conda/envs/my-tensorflow-conda/lib/python3.10/site-pack

In [7]:
training_ds_bag = execution.datasets[0]
testing_ds_bag = execution.datasets[1]
multimodal_full_ds_bag = execution.datasets[2]

crop_image_model = execution.asset_paths[0]

In [8]:
def get_dataframe_from_bag(ds_bag: DatasetBag, multimodal_full_ds_bag: DatasetBag):
    observation_table = ds_bag.get_table_as_dataframe('Observation')
    image_table = ds_bag.get_table_as_dataframe('Image')
    laterality_table = ds_bag.get_table_as_dataframe('Execution_Image_Fundus_Laterality')

    image_table_filtered = image_table[['RID', 'Filename', 'Observation']].rename(columns={'RID': 'RID_Image'})
    laterality_table_filtered = laterality_table[['Image', 'Image_Side']].rename(columns={'Image': 'RID_Image'})
    image_laterality = pd.merge(image_table_filtered, laterality_table_filtered, left_on='RID_Image', right_on='RID_Image', how='inner')
    observation_table_filtered = observation_table[['RID',  'Subject']].rename(columns={'RID': 'RID_Observation'})
    image_laterality_observation = pd.merge(image_laterality, observation_table_filtered, left_on='Observation', right_on='RID_Observation', how='inner')

    wide = EA.multimodal_wide(multimodal_full_ds_bag) 
    
    image_observation_laterality_subject_wide = pd.merge(
     wide, 
     image_laterality_observation, 
     left_on=['RID_Subject', 'Image_Side'], 
     right_on=['Subject', 'Image_Side'], 
     how='inner'
    )

    return image_observation_laterality_subject_wide

In [9]:
train_df = get_dataframe_from_bag(training_ds_bag, multimodal_full_ds_bag)
test_df= get_dataframe_from_bag(testing_ds_bag, multimodal_full_ds_bag)

/home/nguyent8/Repos/eye-ai-ml/eye_ai/eye_ai.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hvf_clean.loc[:, 'priority'] = hvf_clean['Field_Size'].map(priority)
/home/nguyent8/Repos/eye-ai-ml/eye_ai/eye_ai.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hvf_clean.loc[:, 'priority'] = hvf_clean['Field_Size'].map(priority)


In [10]:
from pathlib import Path

# Ensure working_dir is a Path object
working_dir = Path("~/working_dir")

# Save DataFrames as CSV files
train_csv_path = working_dir / "train.csv"
test_csv_path = working_dir / "test.csv"

train_df.to_csv(train_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

In [64]:
table_name = 'Image'
EA.find_features(table_name)

[Feature(target_table=Image, feature_name=Annotation, feature_table=Annotation),
 Feature(target_table=Image, feature_name=Image_Diagnosis, feature_table=Image_Diagnosis),
 Feature(target_table=Image, feature_name=Fundus_Laterality, feature_table=Execution_Image_Fundus_Laterality),
 Feature(target_table=Image, feature_name=Fundus_Angle, feature_table=Execution_Image_Fundus_Angle)]

In [55]:
feature_name = 'Annotation'
Feature = EA.feature_record_class(table_name, feature_name)

In [56]:
from IPython.display import Markdown, display
display(
    Markdown('### Feature Name'),
    [ f'Name: {c.name}, Required: {not c.nullok}' for c in Feature.feature.feature_columns]
)
         

### Feature Name

['Name: Fundus_Bounding_Box, Required: True',
 'Name: Annotation_Function, Required: True',
 'Name: Annotation_Type, Required: True']

In [14]:
cl = [c for c in Feature.feature.feature_columns]
help(cl[0].type)

Help on Type in module deriva.core.ermrest_model object:

class Type(builtins.object)
 |  Type(type_doc)
 |  
 |  Named type.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, type_doc)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  prejson(self, prune=True)
 |  
 |  sqlite3_ddl(self) -> 'str'
 |      Return a SQLite3 column type DDL fragment for this type
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [44]:
# asset_path_models = execution.execution_asset_path("Diagnosis_Model")
asset_path_output = execution.execution_asset_path("Image_Annotation")
asset_path_output

PosixPath('/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S524/execution-asset/Image_Annotation')

In [66]:
bb_csv_path, bb_asset_paths = execution.feature_paths('Image', 'Annotation')
bb_asset_paths

{'Fundus_Bounding_Box': PosixPath('/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S524/feature/eye-ai/Image/Annotation/asset/Fundus_Bounding_Box')}

In [67]:
import shutil
from pathlib import Path

src_dir = Path("/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S524/execution-asset/Image_Annotation")
dst_dir = Path("/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S524/feature/eye-ai/Image/Annotation/asset/Fundus_Bounding_Box")

dst_dir.mkdir(parents=True, exist_ok=True)

for file in src_dir.iterdir():
    if file.is_file():
        shutil.move(str(file), dst_dir / file.name)


In [ ]:
from vgg19_disk_crop_predict import preprocess_and_crop
with execution.execute() as exec:
    preprocess_and_crop(
      multimodal_full_ds_bag,
      '~/working_dir/train.csv',
      '~/working_dir/output/output_train.csv',
      'template.jpg',
      str(asset_path_output),
      crop_image_model,
      "2-NK8E",
      "Optic Nerve",
      False
      )
    
    preprocess_and_crop(
      multimodal_full_ds_bag,
      '~/working_dir/test.csv',
      '~/working_dir/output/output_test.csv',
      'template.jpg',
      str(asset_path_output),
      crop_image_model,
      "2-NK8E",
      "Optic Nerve",
      False
      )

In [20]:
print("checkpoint test 1")

checkpoint test 1


In [45]:
ImageBoundingboxFeature = EA.feature_record_class("Image", feature_name)
ImageBoundingboxFeature

deriva_ml.feature.ImageFeatureAnnotation

In [46]:
image_rids = []
for file_name in os.listdir(asset_path_output):
    image_rids.append(file_name.split("_")[1].split('.')[0])

In [47]:
csv_train = Path("~/working_dir/output/output_train.csv")
df = pd.read_csv(csv_train)

# Create a mapping from Image RID to Worked Image Cropping Function
cropping_func_map_train = dict(zip(df["Image RID"], df["Worked Image Cropping Function"]))

In [48]:
csv_test = Path("~/working_dir/output/output_test.csv")
df = pd.read_csv(csv_test)

# Create a mapping from Image RID to Worked Image Cropping Function
cropping_func_map_test = dict(zip(df["Image RID"], df["Worked Image Cropping Function"]))

In [49]:
# Generate the list
image_bounding_box_feature_list = []
for image_rid in image_rids:
    if (asset_path_output / f"Cropped_{image_rid}.svg").exists():
        if image_rid in cropping_func_map_train:
            image_bounding_box_feature_list.append(
                ImageBoundingboxFeature(
                    Image=image_rid,
                    Execution=execution.execution_rid,
                    Fundus_Bounding_Box=asset_path_output / f"Cropped_{image_rid}.svg",
                    Annotation_Function=cropping_func_map_train.get(image_rid),
                    Annotation_Type='Optic Nerve',
                ))
        if image_rid in cropping_func_map_test:
            image_bounding_box_feature_list.append(
                ImageBoundingboxFeature(
                    Image=image_rid,
                    Execution=execution.execution_rid,
                    Fundus_Bounding_Box=asset_path_output / f"Cropped_{image_rid}.svg",
                    Annotation_Function=cropping_func_map_test.get(image_rid),
                    Annotation_Type='Optic Nerve',
                ))
        

In [ ]:
image_bounding_box_feature_list

ImageFeatureAnnotation(Execution='4-S524', Feature_Name='Annotation', Fundus_Bounding_Box=PosixPath('/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S524/execution-asset/Image_Annotation/Cropped_2-BRVR.svg'), Annotation_Function='imgResize_secondary', Annotation_Type='Optic Nerve', Image='2-BRVR')

In [ ]:
# execution.write_feature_file(image_bounding_box_feature_list)
# execution.upload_execution_outputs()